In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import requests
import geopandas as gpd
import os
import datetime
import time

In [2]:
pd.set_option("display.max_rows", None)

In [3]:
mrt_file_loc = "mrt_stations_raw.csv"
lrt_file_loc = "lrt_stations_raw.csv"
mrt_df = pd.read_csv(mrt_file_loc, parse_dates=['opening'], date_format='%Y-%m-%d')
lrt_df = pd.read_csv(lrt_file_loc, parse_dates=["opening"], date_format="%Y-%m-%d")

In [4]:
df = pd.concat([mrt_df, lrt_df], axis=0, ignore_index=True)
df = df.loc[:, ["alphanumeric_code", "station_name_english", "line_english", "line_color", "opening", "transport_type"]]
df.columns = ["code", "station_name", "line", "color", "opening", "type"]
df

,code,station_name,line,color,opening,type
0,NS1,Jurong East,North-South Line,Red,1990-03-10,MRT
1,NS2,Bukit Batok,North-South Line,Red,1990-03-10,MRT
2,NS3,Bukit Gombak,North-South Line,Red,1990-03-10,MRT
3,NS4,Choa Chu Kang,North-South Line,Red,1990-03-10,MRT
4,NS5,Yew Tee,North-South Line,Red,1996-02-10,MRT
5,NS7,Kranji,North-South Line,Red,1996-02-10,MRT
6,NS8,Marsiling,North-South Line,Red,1996-02-10,MRT
7,NS9,Woodlands,North-South Line,Red,1996-02-10,MRT
8,NS10,Admiralty,North-South Line,Red,1996-02-10,MRT
9,NS11,Sembawang,North-South Line,Red,1996-02-10,MRT


In [5]:
def get_station_geodata(row):
    search_value = row['station_name'] + ' ' + row['type'] + ' station ' + row['code']
    response = requests.get(
        f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={search_value}&returnGeom=Y&getAddrDetails=Y&pageNum=1"
    )
    while response.status_code != 200:
        time.sleep(1)
        response = requests.get(
            f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={search_value}&returnGeom=Y&getAddrDetails=Y&pageNum=1"
        )
    json_data = response.json()['results'][0]
    # print(json_data)

    # search_val = json_data['SEARCHVAL']
    blk_no = json_data['BLK_NO']
    road_name = json_data['ROAD_NAME']
    building = json_data['BUILDING']
    address = json_data['ADDRESS']
    postal = json_data['POSTAL']
    x = json_data['X']
    y = json_data['Y']
    latitude = json_data['LATITUDE']
    longitude = json_data['LONGITUDE']

    return pd.Series([blk_no, road_name, building, address, postal, x, y, latitude, longitude],
                     index=['BLK_NO', 'ROAD_NAME', 'BUILDING', 'ADDRESS', 'POSTAL', 'X', 'Y', 'LATITUDE', 'LONGITUDE'])


In [6]:
df[['blk_no', 'road_name', 'building', 'address', 'postal', 'x', 'y', 'latitude', 'longitude']] = df.apply(get_station_geodata, axis=1)

In [7]:
df.head(5)

,code,station_name,line,color,opening,type,blk_no,road_name,building,address,postal,x,y,latitude,longitude
0,NS1,Jurong East,North-South Line,Red,1990-03-10,MRT,10,JURONG EAST STREET 12,JURONG EAST MRT STATION (EW24 / NS1),10 JURONG EAST STREET 12 JURONG EAST MRT STATI...,609690,17869.0570516568,35038.9688695427,1.33315281585758,103.742286332403
1,NS2,Bukit Batok,North-South Line,Red,1990-03-10,MRT,10,BUKIT BATOK CENTRAL,BUKIT BATOK MRT STATION (NS2),10 BUKIT BATOK CENTRAL BUKIT BATOK MRT STATION...,659958,18679.3223191258,36794.9260214306,1.34903331201636,103.749566478309
2,NS3,Bukit Gombak,North-South Line,Red,1990-03-10,MRT,802,BUKIT BATOK WEST AVENUE 5,BUKIT GOMBAK MRT STATION (NS3),802 BUKIT BATOK WEST AVENUE 5 BUKIT GOMBAK MRT...,659083,18926.9127006577,37854.0358848721,1.35861159094192,103.751790910733
3,NS4,Choa Chu Kang,North-South Line,Red,1990-03-10,MRT,10,CHOA CHU KANG AVENUE 4,CHOA CHU KANG MRT STATION (NS4),10 CHOA CHU KANG AVENUE 4 CHOA CHU KANG MRT ST...,689810,18101.2472041463,40812.1216052417,1.38536316540225,103.744370779756
4,NS5,Yew Tee,North-South Line,Red,1996-02-10,MRT,61,CHOA CHU KANG DRIVE,YEW TEE MRT STATION (NS5),61 CHOA CHU KANG DRIVE YEW TEE MRT STATION (NS...,689715,18438.9831173802,42158.0181021616,1.39753506936297,103.747405150236


In [8]:
github_geo_file_loc = "../../datasets/sg_map/mp2014/MP14_PLNG_AREA_NO_SEA_PL.shp"
geo_gdf = gpd.read_file(github_geo_file_loc)

In [9]:
geo_gdf.head(1)

,OBJECTID,PLN_AREA_N,PLN_AREA_C,CA_IND,REGION_N,REGION_C,INC_CRC,FMEL_UPD_D,X_ADDR,Y_ADDR,SHAPE_Leng,SHAPE_Area,geometry
0,1,ANG MO KIO,AM,N,NORTH-EAST REGION,NER,E5CBDDE0C2113055,2016-05-11,28976.8763,40229.1238,17494.24019,1.394138e+07,"POLYGON ((30658.500 42047.527, 30679.195 42020..."


In [10]:
mrt_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['x'], df['y']), crs=geo_gdf.crs)

In [11]:
mrt_gdf = mrt_gdf.sjoin(geo_gdf[["PLN_AREA_N", "REGION_N", "geometry"]], how='left')

In [12]:
mrt_gdf = mrt_gdf.drop(['geometry', 'index_right'], axis=1).rename(columns={"PLN_AREA_N":"planning_area_ura", "REGION_N":"region_ura"})

In [14]:
mrt_gdf.head()

,code,station_name,line,color,opening,type,blk_no,road_name,building,address,postal,x,y,latitude,longitude,planning_area_ura,region_ura
0,NS1,Jurong East,North-South Line,Red,1990-03-10,MRT,10,JURONG EAST STREET 12,JURONG EAST MRT STATION (EW24 / NS1),10 JURONG EAST STREET 12 JURONG EAST MRT STATI...,609690,17869.0570516568,35038.9688695427,1.33315281585758,103.742286332403,JURONG EAST,WEST REGION
1,NS2,Bukit Batok,North-South Line,Red,1990-03-10,MRT,10,BUKIT BATOK CENTRAL,BUKIT BATOK MRT STATION (NS2),10 BUKIT BATOK CENTRAL BUKIT BATOK MRT STATION...,659958,18679.3223191258,36794.9260214306,1.34903331201636,103.749566478309,BUKIT BATOK,WEST REGION
2,NS3,Bukit Gombak,North-South Line,Red,1990-03-10,MRT,802,BUKIT BATOK WEST AVENUE 5,BUKIT GOMBAK MRT STATION (NS3),802 BUKIT BATOK WEST AVENUE 5 BUKIT GOMBAK MRT...,659083,18926.9127006577,37854.0358848721,1.35861159094192,103.751790910733,BUKIT BATOK,WEST REGION
3,NS4,Choa Chu Kang,North-South Line,Red,1990-03-10,MRT,10,CHOA CHU KANG AVENUE 4,CHOA CHU KANG MRT STATION (NS4),10 CHOA CHU KANG AVENUE 4 CHOA CHU KANG MRT ST...,689810,18101.2472041463,40812.1216052417,1.38536316540225,103.744370779756,CHOA CHU KANG,WEST REGION
4,NS5,Yew Tee,North-South Line,Red,1996-02-10,MRT,61,CHOA CHU KANG DRIVE,YEW TEE MRT STATION (NS5),61 CHOA CHU KANG DRIVE YEW TEE MRT STATION (NS...,689715,18438.9831173802,42158.0181021616,1.39753506936297,103.747405150236,CHOA CHU KANG,WEST REGION


In [15]:
mrt_gdf.shape

(211, 17)

In [16]:
mrt_gdf["station_name"].nunique()

179

In [20]:
today_date = datetime.datetime.today().strftime("%Y-%m-%d")
mrt_gdf.to_csv(f"mrt_lrt_stations_{today_date}.csv")